# Libraries

In [ ]:
## libraries 
import os
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator

#Visualizer
import matplotlib.pyplot as plt

#Neural Network
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf

import time as tm
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
def FullyConnected():
    inputs = Input(shape=(Features.shape[1],), name="input_1")
    layers = Dense(512, activation="selu")(inputs)
    layers = Dense(256, activation="selu")(layers)
    layers = Dense(128, activation="selu")(layers)
    layers = Dense(64, activation="selu")(layers)
    predictions = Dense(len(classes), activation="softmax", name="output_1")(layers)
    model = Model(inputs = inputs, outputs=predictions)
    optimizer=RMSprop() 
    model.compile(optimizer=optimizer,
                loss='categorical_crossentropy', 
                metrics=['accuracy'])
    return model 

In [ ]:
data_path6 = '../Databases/12DB_6FP.csv' 
data_path5 = '../Databases/12DB_5FP.csv' 
data_path3 = '../Databases/12DB_3FP.csv' 
figures_path = './figures' 

In [ ]:
if not os.path.exists(figures_path):
    os.makedirs(figures_path) 
if not os.path.exists(figures_path+"/6FP"):
    os.makedirs(figures_path+"/6FP") 
if not os.path.exists(figures_path+"/5FP"):
    os.makedirs(figures_path+"/5FP") 
if not os.path.exists(figures_path+"/3FP"):
    os.makedirs(figures_path+"/3FP") 

# 6 CLASES

In [ ]:
classes = [0, 1, 2, 3, 4, 5] 

In [ ]:
# Load Dataset
Data = pd.read_csv(data_path6, names=['Vsl', 'Vsg', 'VisL', 'VisG', 'DenL', 'DenG', 'ST', 'Ang', 'ID', 'Flow Pattern'], header=0)
print('Data shape:', Data.shape) 
# Train, test split
features_list = ['Vsl', 'Vsg', 'VisL', 'VisG', 'DenL', 'DenG', 'ST', 'Ang', 'ID']
Features = Data[features_list].values
Labels = Data['Flow Pattern'].values

print('Features shape:', Features.shape)
print('Labels shape:', Labels.shape)

kfold = StratifiedKFold(n_splits=10, random_state=42)
clf = make_pipeline(StandardScaler(), KerasClassifier(FullyConnected, epochs=3, batch_size=64, verbose=0))
start_time = tm.time() 
cv_results = cross_val_score(clf, Features, Labels, cv=kfold, scoring='accuracy')
TIME = tm.time() - start_time 
print("Time FNN = %s [seconds]" % int(TIME))
msg = "%s: %f (%f)" % ('FNN', cv_results.mean(), cv_results.std())
print(msg) 

In [ ]:
cv_results

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Features, Labels, test_size=0.2, stratify=Labels, random_state=42)
print('Train data shape:', X_train.shape)
print('Train labels shape:', y_train.shape)
print('Test data shape:', X_test.shape)
print('Test labels shape:', y_test.shape)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X_train_ = np.concatenate([X_train,X_test],axis=0)
y_train_ = np.concatenate([y_train,y_test],axis=0)
print('Data shape:', X_train_.shape)
print('labels shape:', y_train_.shape) 

models = [('FNN', KerasClassifier(FullyConnected, epochs=400, batch_size=64, verbose=0))]

# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=42)
    start_time = tm.time() 
    cv_results = cross_val_score(model, X_train_, y_train_, cv=kfold, scoring='accuracy')
    TIME = tm.time() - start_time 
    print("Time "+str(name)+" = %s [seconds]" % int(TIME))
    results.append(cv_results)
    names.append(name) 
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# Compare Algorithms
with plt.style.context('seaborn-white'):
    fig = plt.figure(figsize=(10, 10)) 
    ax = fig.add_subplot(111) 
    plt.boxplot(results) 
    ax.set_xticklabels(names)
    plt.savefig(figures_path+"/6FP/"+str(len(classes))+"_cross_valid.pdf", format='pdf')
    plt.show() 

# 5 CLASES

In [ ]:
classes = [0, 1, 2, 3, 4] 

In [ ]:
# Load Dataset 
Data = pd.read_csv(data_path5, names=['Vsl', 'Vsg', 'VisL', 'VisG', 'DenL', 'DenG', 'ST', 'Ang', 'ID', 'Flow Pattern'], header=0)
print('Data shape:', Data.shape)
# Train, test split
features_list = ['Vsl', 'Vsg', 'VisL', 'VisG', 'DenL', 'DenG', 'ST', 'Ang', 'ID']
Features = Data[features_list]
Labels = Data['Flow Pattern']
X_train, X_test, y_train, y_test = train_test_split(Features, Labels, test_size=0.2, stratify=Labels, random_state=42)
print('Train data shape:', X_train.shape)
print('Train labels shape:', y_train.shape)
print('Test data shape:', X_test.shape)
print('Test labels shape:', y_test.shape)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train_ = np.concatenate([X_train,X_test],axis=0)
y_train_ = np.concatenate([y_train,y_test],axis=0)
print('Data shape:', X_train_.shape)
print('labels shape:', y_train_.shape) 

In [ ]:
# Spot Check Algorithms
models = [('FNN', KerasClassifier(FullyConnected, epochs=3, batch_size=64, verbose=0))]

# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=42)
    start_time = tm.time() 
    cv_results = cross_val_score(model, X_train_, y_train_, cv=kfold, scoring='accuracy')
    TIME = tm.time() - start_time 
    print("Time "+str(name)+" = %s [seconds]" % int(TIME))
    results.append(cv_results)
    names.append(name) 
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# Compare Algorithms
with plt.style.context('seaborn-white'):
    fig = plt.figure(figsize=(10, 10)) 
    ax = fig.add_subplot(111) 
    plt.boxplot(results) 
    ax.set_xticklabels(names)
    plt.savefig(figures_path+"/5FP/"+str(len(classes))+"_cross_valid.pdf", format='pdf')
    plt.show() 

# 3 CLASES

In [ ]:
classes = [0, 1, 2] 

In [ ]:
# Load Dataset
Data = pd.read_csv(data_path3, names=['Vsl', 'Vsg', 'VisL', 'VisG', 'DenL', 'DenG', 'ST', 'Ang', 'ID', 'Flow Pattern'], header=0)
print('Data shape:', Data.shape)
# Train, test split
features_list = ['Vsl', 'Vsg', 'VisL', 'VisG', 'DenL', 'DenG', 'ST', 'Ang', 'ID']
Features = Data[features_list]
Labels = Data['Flow Pattern']
X_train, X_test, y_train, y_test = train_test_split(Features, Labels, test_size=0.2, stratify=Labels, random_state=42)
print('Train data shape:', X_train.shape)
print('Train labels shape:', y_train.shape)
print('Test data shape:', X_test.shape)
print('Test labels shape:', y_test.shape)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train_ = np.concatenate([X_train,X_test],axis=0)
y_train_ = np.concatenate([y_train,y_test],axis=0)
print('Data shape:', X_train_.shape)
print('labels shape:', y_train_.shape) 

In [ ]:
# Spot Check Algorithms
models = [('FNN', KerasClassifier(FullyConnected, epochs=1, batch_size=64, verbose=0))]

# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=42)
    start_time = tm.time() 
    cv_results = cross_val_score(model, X_train_, y_train_, cv=kfold, scoring='accuracy')
    TIME = tm.time() - start_time 
    print("Time "+str(name)+" = %s [seconds]" % int(TIME))
    results.append(cv_results)
    names.append(name) 
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# Compare Algorithms 
with plt.style.context('seaborn-white'):
    fig = plt.figure(figsize=(10, 10)) 
    ax = fig.add_subplot(111) 
    plt.boxplot(results) 
    ax.set_xticklabels(names)
    plt.savefig(figures_path+"/3FP/"+str(len(classes))+"_cross_valid.pdf", format='pdf')
    plt.show() 